In [1]:
import numpy as np
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector

# --- 1. Build a VQE-like Ansatz Circuit (2-qubit) ---
ansatz = TwoLocal(num_qubits=2, reps=1, rotation_blocks=['ry', 'rz'], entanglement_blocks='cx')
'''
TwoLocal ansatz: This creates a parameterized circuit with 2 qubits, using RY and RZ rotations and CNOT 
as entangling gates. This is a common circuit template for variational algorithms like VQE.
'''

# Random or optimized parameters (for test, use random)
params = np.random.uniform(0, 2 * np.pi, ansatz.num_parameters)
circuit = ansatz.assign_parameters(params)
'''
Random parameters: Fills in the ansatz’s rotation parameters with random (or, in real VQE, optimized) 
values, making a concrete circuit.
'''
print(circuit)

     ┌────────────────────────────────────────────────────────────────────┐
q_0: ┤0                                                                   ├
     │  TwoLocal(3.2512,1.0881,1.5035,0.54645,2.8557,3.834,1.2947,6.2014) │
q_1: ┤1                                                                   ├
     └────────────────────────────────────────────────────────────────────┘


In [2]:
# --- 2. Simulate Quantum Statevector of Circuit ---
output_state = Statevector.from_instruction(circuit) # 
probs = output_state.probabilities_dict()    # {'00': p0, '01': p1, ...}
'''
Simulate quantum state: Calculates the exact quantum state (statevector) after applying the quantum circuit.
Convert to probabilities: Gets probabilities for each bitstring output (e.g. '00', '01', etc.).
'''

# --- 3. Reference / ideal distribution: for example, |00> state
ideal_state = Statevector.from_label('00')
ideal_probs = ideal_state.probabilities_dict()
'''
Set ideal state: Here, it uses the computational basis state |00⟩ as the perfect reference state.
Get ideal probabilities: For |00⟩, this is { '00': 1.0 }.
'''

# --- 4. Hellinger (Supermarq-style) Fidelity Function ---
def polarization_fidelity(p, q):
    keys = set(p.keys()) | set(q.keys())
    return (sum(np.sqrt(p.get(k, 0.0) * q.get(k, 0.0)) for k in keys)) ** 2
'''
Supermarq hellinger fidelity: Measures similarity between two probability distributions (measured vs ideal).
The value is in ; 1 means the distributions are identical, 0 means fully different.
'''
# --- 5. Calculate and Print Fidelity ---

'''
Calculates the fidelity between your real (possibly noisy/optimized) distribution and the "ideal" target.
Prints the result; closer to 1 means closer to the ideal state.
'''

fidelity = polarization_fidelity(probs, ideal_probs)
print("Supermarq-style VQE fidelity:", fidelity)




Supermarq-style VQE fidelity: 0.9023051961615187


Generalised Code is below: 

In [3]:
import numpy as np
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector

def run_vqe_fidelity(num_qubits):
    # Create the TwoLocal variational ansatz with specified number of qubits
    ansatz = TwoLocal(
        num_qubits=num_qubits,
        reps=1,  # number of repetitions of rotation + entanglement layers
        rotation_blocks=['ry', 'rz'],  # parameterized rotation gates
        entanglement_blocks='cx'       # CNOT gates for entanglement
    )
    
    # Generate random parameter values for the ansatz
    params = np.random.uniform(0, 2 * np.pi, ansatz.num_parameters)
    
    # Assign parameters to create a concrete circuit
    circuit = ansatz.assign_parameters(params)
    
    # Optional: Print the circuit structure
    print(f"VQE Ansatz circuit for {num_qubits} qubits:")
    print(circuit.draw())
    
    # Simulate the final quantum statevector from the circuit
    output_state = Statevector.from_instruction(circuit)
    probs = output_state.probabilities_dict()  # Get output probabilities dictionary
    
    # Define the ideal ground state as |0...0> computational basis vector
    ideal_label = '0' * num_qubits
    ideal_state = Statevector.from_label(ideal_label)
    ideal_probs = ideal_state.probabilities_dict()
    
    # Supermarq-style Hellinger fidelity function between two distributions
    def polarization_fidelity(p, q):
        keys = set(p) | set(q)
        return (sum(np.sqrt(p.get(k, 0.0) * q.get(k, 0.0)) for k in keys)) ** 2
    
    # Compute and print the fidelity
    fidelity = polarization_fidelity(probs, ideal_probs)
    print(f"Supermarq-style VQE fidelity for {num_qubits} qubits:", fidelity)
    return fidelity

# Example: Run for 3 qubits (or any number you want)
run_vqe_fidelity(3)


VQE Ansatz circuit for 3 qubits:
     »
q_0: »
     »
q_1: »
     »
q_2: »
     »
«     ┌────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                   ├
«     │                                                                                                    │
«q_1: ┤1 TwoLocal(0.80611,6.1537,0.3807,0.97568,2.4367,0.70169,4.5783,4.9133,6.1607,0.98239,3.3708,1.1318) ├
«     │                                                                                                    │
«q_2: ┤2                                                                                                   ├
«     └────────────────────────────────────────────────────────────────────────────────────────────────────┘
Supermarq-style VQE fidelity for 3 qubits: 0.3536335229450583


0.3536335229450583